<a href="https://colab.research.google.com/github/jhodgens17/audio_classification/blob/main/MVP_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import packages
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
target_words = ['down', 'go', 'left', 'no', 'off',
           'on', 'right', 'stop', 'up', 'yes']

In [ ]:
unknown = ['bed', 'bird', 'cat', 'dog',  
           'eight', 'five', 'four', 'happy', 
           'house', 'marvin', 'nine', 'one', 'seven',
           'sheila', 'six', 'three', 'tree', 'two',
           'wow', 'zero']

In [ ]:
all_words = target_words + unknown

In [ ]:
len(all_words)

30

In [ ]:
all_files = {}
for word in all_words:
  path = '/content/drive/MyDrive/train/audio/'+word
  file_list = os.listdir(path)
  all_files[word] = file_list

In [ ]:
len(all_files)

30

In [ ]:
for word in all_files:
  print(len(all_files[word]))

2359
2372
2353
2375
2357
2367
2367
2380
2375
2377
1713
1731
1733
1746
2352
2357
2372
1742
1750
1746
2364
2370
2377
1734
2369
2356
1733
2373
1745
2376


In [ ]:
total = 0
for word in all_files:
  total = total + len(all_files[word])

In [ ]:
total

64721

In [ ]:
all_paths = {}
for word in all_files:
  paths = []
  for file in all_files[word]:
    path = '/content/drive/MyDrive/train/audio/' + word + '/' + file
    paths.append(path)
  all_paths[word] = paths

In [ ]:
len(all_paths)

30

In [ ]:
total_paths = 0
for word in all_paths:
  total_paths = total_paths + len(all_paths[word])

In [ ]:
total_paths

64721

In [ ]:
all_paths = []
for target in all_files:
  for file in all_files[target]:
    path = '/content/drive/MyDrive/train/audio/' + target + '/' + file
    all_paths.append(path)

In [ ]:
len(all_paths)

64721

In [ ]:
all_paths[0]

'/content/drive/MyDrive/train/audio/down/be91a00a_nohash_1.wav'

In [ ]:
#Defining a function to get the label for each audio clip
def get_label(file_path):
  parts = tf.strings.split(file_path, sep = '/')
  return parts[-2]

In [ ]:
#Defining function to get waveform for each clip
def get_waveform(path):
  audio_binary = tf.io.read_file(path)
  audio, sample_rate = tf.audio.decode_wav(audio_binary)
  waveform = tf.squeeze(audio, axis=-1)
  return waveform

In [ ]:
#Defining a function to get the waveform and label
def get_waveform_and_label(file_path):
  label = get_label(file_path)
  waveform = get_waveform(file_path)
  return waveform, label

In [ ]:
#Defining a function that makes all audio clips same length and gets spectrogram
def get_spectrogram(waveform):
  zero_padding = tf.zeros([16000] - tf.shape(waveform), dtype=tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(equal_length, frame_length=255, frame_step=128)
  spectrogram = tf.abs(spectrogram)
  return spectrogram

In [ ]:
get_label('/content/drive/MyDrive/train/audio/seven/264f471d_nohash_3.wav').numpy()

b'seven'

In [ ]:
target_words_updated = ["b'down", "b'go'", "b'left'", "b'no'", "b'off'", 
                        "b'on'", "b'right'", "b'stop'", "b'up'", "b'yes'"]

In [ ]:
def get_label_id(label):
  if label.numpy() == b'down':
    return [1,0,0,0,0,0,0,0,0,0,0]
  elif label.numpy() == b'go':
    return [0,1,0,0,0,0,0,0,0,0,0]
  elif label.numpy() == b'left':
    return [0,0,1,0,0,0,0,0,0,0,0]
  elif label.numpy() == b'no':
    return [0,0,0,1,0,0,0,0,0,0,0]
  elif label.numpy() == b'off':
    return [0,0,0,0,1,0,0,0,0,0,0]
  elif label.numpy() == b'on':
    return [0,0,0,0,0,1,0,0,0,0,0]
  elif label.numpy() == b'right':
    return [0,0,0,0,0,0,1,0,0,0,0]
  elif label.numpy() == b'stop':
    return [0,0,0,0,0,0,0,1,0,0,0]
  elif label.numpy() == b'up':
    return [0,0,0,0,0,0,0,0,1,0,0]
  elif label.numpy() == b'yes':
    return [0,0,0,0,0,0,0,0,0,1,0]
  else:
    return [0,0,0,0,0,0,0,0,0,0,1]

In [ ]:
lab_test = get_label('/content/drive/MyDrive/train/audio/seven/264f471d_nohash_3.wav')

In [ ]:
get_label_id(lab_test)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
#Defining funciton to get spectrogram and label id
def get_spectrogram_and_label_id(waveform, label):
  spectrogram = get_spectrogram(waveform)
  label_id = get_label_id(label)
  return spectrogram, label_id

In [ ]:
#Getting all of the files
files_ds = tf.data.Dataset.from_tensor_slices(all_paths)

In [ ]:
#Getting the waveform and label for each file
waveform_ds = files_ds.map(get_waveform_and_label)

In [ ]:
#Getting the spectrogram for each file
spectrogram_ds = waveform_ds.map(get_spectrogram_and_label_id)

AttributeError: ignored